In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

# Definir un User-Agent personalizado
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"

# Configurar las opciones de Chrome
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={user_agent}')
options.add_argument("--disable-gpu")  # Desactiva la GPU para ahorrar recursos
options.add_argument("--no-sandbox")  # Previene errores en algunos sistemas
options.add_argument("--disable-extensions")  # Desactiva las extensiones para mejorar la velocidad
options.add_argument("--disable-images")  # Desactiva las imágenes para mejorar la velocidad
options.add_argument("--start-maximized")
options.headless = True  # Ejecuta el navegador en modo headless (sin interfaz gráfica)

# Iniciar el navegador con las opciones configuradas
driver = webdriver.Chrome(options=options)

In [2]:
# Navegar a la página de Pluto.tv
driver.get('https://pluto.tv/latam/on-demand')

# Espera que la página cargue completamente (ajusta según sea necesario)
driver.implicitly_wait(10)

time.sleep(5)

In [3]:
# Encuentra y haz clic en el span "Películas"
span_peliculas = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//span[text()='Películas']")))
span_peliculas.click()

# Espera a que se cargue la sección de películas (ajusta según sea necesario)
time.sleep(5)

In [4]:
def process_ul_after_peliculas():
    hrefs = []
    ul_counter = 0  # Contador de ul procesados
    max_scrolls = 50  # Aumentar el límite de desplazamientos
    scrolls_done = 0  # Contador de desplazamientos realizados
    
    try:
        # Encuentra el h3 con el texto "Películas"
        start_h3 = driver.find_element(By.XPATH, "//h3[text()='Películas']")
        print(f"Encontrado h3: Películas")
        
        # Inicia la búsqueda después del h3 "Películas"
        current_element = start_h3
        found_retro = False
        
        while scrolls_done < max_scrolls:
            try:
                ul_element = current_element.find_element(By.XPATH, "./following::ul[1]")  # Encuentra el primer ul después del h3
                
                # Procesa el ul actual
                hrefs.extend(extract_hrefs_from_ul(ul_element))
                ul_counter += 1
                
                print(f"Procesado ul {ul_counter}")
                
                # Actualiza current_element para continuar después del ul procesado
                current_element = ul_element
                
                if found_retro:
                    # Continúa procesando 6 ul más después de encontrar "Retro"
                    if ul_counter >= 6:
                        print("Se procesaron 6 ul adicionales después de 'Retro'. Terminando la extracción.")
                        break
                
                else:
                    # Verifica si hay un h3 con el texto 'Retro' después del current_element
                    try:
                        retro_h3 = driver.find_element(By.XPATH, "//h3[contains(text(), 'Retro')]")
                        print("Encontrado h3 con texto 'Retro'. Continuando con la extracción de 6 ul más.")
                        found_retro = True
                        ul_counter = 0  # Reiniciar el contador para procesar 6 ul más
                    except:
                        pass

                # Mueve el foco al ul actual antes de hacer scroll
                driver.execute_script("arguments[0].scrollIntoView();", ul_element)
                print(f"Foco en ul actual antes de AvPag.")
                
                # Presiona AvPag para continuar cargando contenido
                press_page_down(1)
                scrolls_done += 1
                
            except Exception as e:
                print(f"No se encontraron más ul después de 'Películas' o se alcanzó 'Retro'. Error: {e}")
                break
    
    except Exception as e:
        print(f"Error al encontrar el h3 'Películas': {e}")
    
    return hrefs

# Función para presionar AvPag
def press_page_down(times):
    actions = ActionChains(driver)
    for _ in range(times):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(2)  # Pausa para permitir la carga del contenido

# Función para extraer hrefs de un ul
def extract_hrefs_from_ul(ul_element):
    hrefs = []
    li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
    for li in li_elements:
        a_tag = li.find_element(By.TAG_NAME, 'a')
        hrefs.append(a_tag.get_attribute('href'))
    return hrefs

# Ejecuta la función para extraer los hrefs
peliculas_hrefs = process_ul_after_peliculas()

# Imprime las URLs extraídas
print(f"Películas URLs: {peliculas_hrefs}")

# Cierra el navegador
driver.quit()


Encontrado h3: Películas
Procesado ul 1
Foco en ul actual antes de AvPag.
Procesado ul 2
Foco en ul actual antes de AvPag.
Procesado ul 3
Foco en ul actual antes de AvPag.
Procesado ul 4
Foco en ul actual antes de AvPag.
Procesado ul 5
Foco en ul actual antes de AvPag.
Procesado ul 6
Foco en ul actual antes de AvPag.
Procesado ul 7
Foco en ul actual antes de AvPag.
Procesado ul 8
Foco en ul actual antes de AvPag.
Procesado ul 9
Foco en ul actual antes de AvPag.
Procesado ul 10
Foco en ul actual antes de AvPag.
Procesado ul 11
Foco en ul actual antes de AvPag.
Procesado ul 12
Foco en ul actual antes de AvPag.
Procesado ul 13
Foco en ul actual antes de AvPag.
Procesado ul 14
Foco en ul actual antes de AvPag.
Procesado ul 15
Foco en ul actual antes de AvPag.
Procesado ul 16
Foco en ul actual antes de AvPag.
Procesado ul 17
Foco en ul actual antes de AvPag.
Procesado ul 18
Foco en ul actual antes de AvPag.
Procesado ul 19
Foco en ul actual antes de AvPag.
Procesado ul 20
Foco en ul actual 

In [23]:
peliculas_hrefs

['https://pluto.tv/on-demand/series/645ab1ce5348f4001a21019c/details',
 'https://pluto.tv/on-demand/movies/5f3fd896ad190c001a52e6da/details',
 'https://pluto.tv/on-demand/movies/5fad4a7f8b78ef001aa27bcd/details',
 'https://pluto.tv/on-demand/movies/62bcab10e9e8c00013709d5a/details',
 'https://pluto.tv/on-demand/movies/60c8abd436537d0013320af4/details',
 'https://pluto.tv/on-demand/movies/62bf09238672a30013bd7ef8/details',
 'https://pluto.tv/on-demand/movies/603e6222049634001a5f3418/details',
 'https://pluto.tv/on-demand/movies/60c8abc136537d0013320966/details',
 'https://pluto.tv/on-demand/movies/5fda4095aabcdd001aaf4430/details',
 'https://pluto.tv/on-demand/movies/5fda414b0dddaf0013b83734/details',
 'https://pluto.tv/on-demand/movies/62ffebd0c24630001a8b226c/details',
 'https://pluto.tv/on-demand/movies/62fe9e866df8ae001a0464ae/details',
 'https://pluto.tv/on-demand/movies/60e459453988a50013b7617f/details',
 'https://pluto.tv/on-demand/movies/61f2a44d04ff25001a4fe790/details',
 'http

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar opciones de Selenium
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-extensions")
options.add_argument("--disable-images")
options.add_argument("--start-maximized")
options.headless = True

# Iniciar el navegador con las opciones configuradas
driver = webdriver.Chrome(options=options)

def check_links_status_with_selenium(links):
    status_results = []
    
    for index, link in enumerate(links):
        try:
            print(f"Accediendo al link {index + 1}/{len(links)}: {link}")
            
            # Navega a la URL
            driver.get(link)
            
            # Espera a que aparezca el div con id 'overlay-container' o la clase 'overlayContainer-0-2-15'
            try:
                overlay_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located(
                        (By.ID, 'overlay-container')
                    )
                )

            except:
                try:
                    overlay_element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located(
                            (By.CLASS_NAME, 'overlayContainer-0-2-15')
                        )
                    )

                except Exception as e:
                    print(f"No se encontró el overlay. Saltando al siguiente link. Error: {e}")
                    continue
                
            time.sleep(8)
            
            #TITLE
            
            try:
                header_element = overlay_element.find_element(By.TAG_NAME, 'header')
                h1_element = header_element.find_element(By.TAG_NAME, 'h1')
                h1_text = h1_element.text
                print(f"Texto del h1: {h1_text}")
            except Exception as e:
                h1_text = f"No se pudo encontrar el h1: {e}"
           
            #DESCRIPTION
                
            div_inner = driver.find_element(By.CSS_SELECTOR,'div.inner')
             
            p_element = div_inner.find_element(By.TAG_NAME,'p')
            description_p = p_element.text
            
            print(description_p)

            #INFORMATION
            
            info_inner_lis = div_inner.find_elements(By.TAG_NAME,'li')
            li_texts = [li.text for li in info_inner_lis]
            print(li_texts)
            
            status_results.append({
                'Url': link,
                'Title': h1_text,
                'Resume': description_p,
                'Information': li_texts,
            })
            print(f"Información extraída del link {index + 1}/{len(links)}")
        
        except Exception as e:
            status_results.append({
                'url': link,
                'title': 'Error',
                'description': str(e)
            })
            print(f"Error al acceder al link {index + 1}/{len(links)}: {e}")
    
    return status_results

# Ejecuta la función con las URLs almacenadas en peliculas_hrefs
status_results = check_links_status_with_selenium(peliculas_hrefs)

# Imprime los resultados
for result in status_results:
    print(result)

# Cierra el navegador
driver.quit()


Accediendo al link 1/502: https://pluto.tv/on-demand/series/645ab1ce5348f4001a21019c/details
Texto del h1: Pet Obsessed
Con clips ridículos, cortos digitales y toneladas de pelusa; estas son las mascotas más escandalosas y francas de Internet... Tu nueva obsesión.
['TV-G', '', 'Noticias e Información', '', '', '', '4 Seasons Available', 'Doggo Lingo\nDogos, cachorros, parlanchines y aburridos: ¡explicado!\nS1E10124 min', 'Mischief Makers\nLo bueno, lo malo y lo muy, muy travieso.\nS1E10224 min', 'Spaww Day\nSiéntese, relájese y deléitese con estos increíbles clips.\nS1E10324 min', 'Workaholics\nHa sido un duro día de noche, y han estado trabajando como un perro\nS1E10424 min', 'Here They Pun!\nJuegos de palabras con patitas\nS1E10524 min', 'Sweet and Sassy\nDulce por fuera, atrevida por dentro\nS1E10624 min', 'PET-ducate Yourself\nAbre tu libro de texto Pet 101 y prepárate para el test del cachorro\nS1E10724 min', 'Four Paw Entertainment\nEstas mascotas merecen llevarse a casa un hueso

KeyboardInterrupt: 

In [7]:
# def process_ul_after_peliculas():
#     hrefs = []
#     ul_counter = 0  # Contador de ul procesados
#     max_scrolls = 27  # Límite de desplazamientos para evitar un bucle infinito
#     scrolls_done = 0  # Contador de desplazamientos realizados
    
#     try:
#         # Encuentra el h3 con el texto "Películas"
#         start_h3 = driver.find_element(By.XPATH, "//h3[text()='Películas']")
#         print(f"Encontrado h3: Películas")
        
#         # Inicia la búsqueda después del h3 "Películas"
#         current_element = start_h3
        
#         while scrolls_done < max_scrolls:
#             # Busca el siguiente ul que aparece después del h3
#             try:
#                 ul_element = current_element.find_element(By.XPATH, "./following::ul[1]")  # Encuentra el primer ul después del h3
                
#                 # Procesa el ul actual
#                 hrefs.extend(extract_hrefs_from_ul(ul_element))
#                 ul_counter += 1
                
#                 print(f"Procesado ul {ul_counter}")
                
#                 # Actualiza current_element para continuar después del ul procesado
#                 current_element = ul_element
                
#                 # Verifica si hay un h3 con el texto 'Retro' después del current_element
#                 try:
#                     retro_h3 = driver.find_element(By.XPATH, "//h3[contains(text(), 'Retro')]")
#                     print("Encontrado h3 con texto 'Retro'. Terminando la extracción.")
#                     break
#                 except:
#                     pass

#                 # Mueve el foco al ul actual antes de hacer scroll
#                 driver.execute_script("arguments[0].scrollIntoView();", ul_element)
#                 print(f"Foco en ul actual antes de AvPag.")

#                 # Presiona AvPag para continuar cargando contenido
#                 press_page_down(1)
#                 scrolls_done += 1
                
#             except Exception as e:
#                 print(f"No se encontraron más ul después de 'Películas' o se alcanzó 'Retro'. Error: {e}")
#                 break
    
#     except Exception as e:
#         print(f"Error al encontrar el h3 'Películas': {e}")
    
#     return hrefs

# # Función para presionar AvPag
# def press_page_down(times):
#     actions = ActionChains(driver)
#     for _ in range(times):
#         actions.send_keys(Keys.PAGE_DOWN).perform()
#         time.sleep(2)  # Pausa para permitir la carga del contenido

# # Función para extraer hrefs de un ul
# def extract_hrefs_from_ul(ul_element):
#     hrefs = []
#     li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
#     for li in li_elements:
#         a_tag = li.find_element(By.TAG_NAME, 'a')
#         hrefs.append(a_tag.get_attribute('href'))
#     return hrefs

# # Ejecuta la función para extraer los hrefs
# peliculas_hrefs = process_ul_after_peliculas()

# # Imprime las URLs extraídas
# print(f"Películas URLs: {peliculas_hrefs}")

# # Cierra el navegador
# driver.quit()
